In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
! pip install pdbpp
! pip install pycocoevalcap
! pip install PyDictionary
! pip install nltk==3.6.5
! pip inSstall torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-gc1hut4h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-gc1hut4h
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=c6bb9ed118a9053f5bcb2b109ddad2db5be2f5ec97a03ef767144c81433c34cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-3u025mwc/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=a0992973b73380fad6e010aeff6b188248ca9ac8d7512a947d598a0170de07f0
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for goslate: filename=goslate-1.5.4-py3-none-any.whl size=11594 sha256=792709d531b5fb87e4baba43cbd15b6c4b928f255cc9f924bb4a7632b22908ae
  Stored in directory: /root/.cache/pip/wheels/b5/30/e9/63b6de83667be2977ee793a146a2c80f8e588d5c0203b39dc9
  Created wheel for futures: filename=futures-3.0.5-py3-none-any.whl size=14087 sha256=92

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
ERROR: unknown command "inSstall" - maybe you meant "install"


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_362"
OpenJDK Runtime Environment (build 1.8.0_362-8u362-ga-0ubuntu1~20.04.1-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


In [3]:
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice

from google.colab import drive
drive.mount('/content/gdrive')

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import numpy as np
import torch
from PIL import Image
import pandas as pd
import random
import copy
import os
from tqdm import tqdm

Mounted at /content/gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
import clip

# RN50 will be fastest, while ViT-L will be the most performant.
clip.available_models()


clip_backbones = {'ResNet50': "RN50", 'ResNet101': "RN50x16", 'ViTL': 'ViT-L/14', 'ViTB': "ViT-B/16", 'ViTB32': 'ViT-B/32'}

chosen_model = 'ViTL'

clip_model, preprocessor = clip.load(clip_backbones[chosen_model])
clip_model.cuda().eval()

input_resolution = clip_model.visual.input_resolution
context_length = clip_model.context_length
vocab_size = clip_model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|████████████████████████████████████████| 890M/890M [00:06<00:00, 136MiB/s]


Model parameters: 427,616,513
Input resolution: 224
Context length: 77
Vocab size: 49408


In [ ]:
cider_metric = Cider()
spice_metric = Spice()

Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.


In [ ]:
def get_weights(max_n=1):

  if max_n==1:
    weights = (1, 0, 0, 0)
  elif max_n == 2:
    weights = (0.5, 0.5, 0, 0)
  elif max_n == 3:
    weights = (0.33, 0.33, 0.33, 0)
  else:
    weights = (0.25, 0.25, 0.25, 0.25)

  return weights 

def compute_all_bleu_scores(caption_dictionary, experiment='test'):
  
  bleu_df = pd.read_csv('gdrive/Shareddrives/CS2952N/human_preference.csv')
  
  avg_bleu_scores = []

  all_hypoth_captions = []
  all_gt_captions = []

  all_bleu1 = []
  all_bleu2 = []
  all_bleu3 = []
  all_bleu4 = []

  references = []
  hypotheses = []

  for idx, row in bleu_df.iterrows():

    if row['image_filename'] not in expert_annotations:
      continue

    hypoth_caption = caption_dictionary[row['target_caption']][2]
    gt_captions = caption_dictionary[row['gt_captions']]

    all_hypoth_captions.append(hypoth_caption)
    all_gt_captions.append(';'.join(gt_captions))


    hypotheses.append(hypoth_caption.split(' '))
    ref_for_i = []

    for i in range(len(gt_captions)):

      ref_for_i.append(gt_captions[i].split(' '))
      
    references.append(ref_for_i)

  
  for n in range(1,5):
   
    weights = get_weights(n)


    for ref,hyp in zip(references, hypotheses):
      bleu_score = sentence_bleu(ref, hyp, weights=weights)

      if n==1:
        all_bleu1.append(bleu_score)
      elif n==2:
        all_bleu2.append(bleu_score)
      elif n==3:
        all_bleu3.append(bleu_score)
      elif n==4:
        all_bleu4.append(bleu_score)

      
      

    corp_bleu = corpus_bleu(references, hypotheses, weights = weights)
    avg_bleu_scores.append(corp_bleu)
  

  bleu_df['bleu1_score'] = all_bleu1
  bleu_df['bleu2_score'] = all_bleu2
  bleu_df['bleu3_score'] = all_bleu3
  bleu_df['bleu4_score'] = all_bleu4
  bleu_df['target_caption_text'] = all_hypoth_captions
  bleu_df['gt_caption_text'] = all_gt_captions
 
  bleu_df.to_csv('gdrive/Shareddrives/CS2952N/human_preference.csv', index=False)
  
  return avg_bleu_scores 

def compute_meteor_score(gt_captions, pred_caption):
  """
  pred_caption: string containing predicted caption
  gt_caption: string containing ground-truth caption
  """

  return max([meteor_score.single_meteor_score(word_tokenize(gt), word_tokenize(pred_caption)) for gt in gt_captions])



def compute_all_meteor_scores(caption_dictionary, experiment='test'):

  meteor_df = pd.read_csv('gdrive/Shareddrives/CS2952N/human_preference.csv')
  

  avg_meteor = 0
  count = 0

  all_meteor_scores = []
  all_hypoth_captions = []
  all_gt_captions = []

  for idx, row in meteor_df.iterrows():

    if row['image_filename'] not in expert_annotations:
      continue

    hypoth_caption = caption_dictionary[row['target_caption']][2]
    gt_captions = caption_dictionary[row['gt_captions']]

    all_hypoth_captions.append(hypoth_caption)
    all_gt_captions.append(';'.join(gt_captions))


    refs = []
    for i in range(len(gt_captions)):

      refs.append(gt_captions[i])

    score = compute_meteor_score(refs, hypoth_caption)
      
    all_meteor_scores.append(score)

    avg_meteor += score
    count += 1

  meteor_df['target_caption_text'] = all_hypoth_captions
  meteor_df['gt_caption_text'] = all_gt_captions
  meteor_df['meteor_score'] = all_meteor_scores
  meteor_df.to_csv('gdrive/Shareddrives/CS2952N/human_preference.csv', index=False)
  
  return avg_meteor/count



def compute_all_spice_scores(caption_dictionary, experiment='test'):
  
  spice_df = pd.read_csv('gdrive/Shareddrives/CS2952N/human_preference.csv')

  references = {}
  hypotheses = {}

  all_spice_scores = []
  all_hypoth_captions = []
  all_gt_captions = []


  for idx, row in spice_df.iterrows():

    if row['image_filename'] not in expert_annotations:
      continue

    hypoth_caption = caption_dictionary[row['target_caption']][2]
    gt_captions = caption_dictionary[row['gt_captions']]

    all_hypoth_captions.append(hypoth_caption)
    all_gt_captions.append(';'.join(gt_captions))


    hypotheses[row['target_caption']+'_'+row['gt_captions']] = [hypoth_caption]
    references[row['target_caption']+'_'+row['gt_captions']] = gt_captions
  

  avg_score, scores = spice_metric.compute_score(references, hypotheses)

  sorted_keys = list(sorted(hypotheses.keys()))
  sorted_indexes = {sorted_keys[i]:i for i in range(len(sorted_keys))}


  all_spice_scores = []

  for _, row in spice_df.iterrows():
    idx = sorted_indexes[row['target_caption'] + '_' + row['gt_captions']]
    all_spice_scores.append(float(scores[idx]['All']['f']))
  
  spice_df['target_caption_text'] = all_hypoth_captions
  spice_df['gt_caption_text'] = all_gt_captions
  spice_df['spice_score'] = all_spice_scores
  spice_df.to_csv('gdrive/Shareddrives/CS2952N/human_preference.csv', index=False)
  return avg_score


def compute_all_cider_scores(caption_dictionary, experiment='test'):
  
  cider_df = pd.read_csv('gdrive/Shareddrives/CS2952N/human_preference.csv')

  references = {}
  hypotheses = {}

  all_cider_scores = []
  all_hypoth_captions = []
  all_gt_captions = []


  for idx, row in cider_df.iterrows():

    if row['image_filename'] not in expert_annotations:
      continue

    hypoth_caption = caption_dictionary[row['target_caption']][2]
    gt_captions = caption_dictionary[row['gt_captions']]

    all_hypoth_captions.append(hypoth_caption)
    all_gt_captions.append(';'.join(gt_captions))


    hypotheses[row['target_caption']+'_'+row['gt_captions']] = [hypoth_caption]
    references[row['target_caption']+'_'+row['gt_captions']] = gt_captions
  

  avg_score, scores = cider_metric.compute_score(references, hypotheses)

  sorted_keys = list(sorted(hypotheses.keys()))
  sorted_indexes = {sorted_keys[i]:i for i in range(len(sorted_keys))}


  all_cider_scores = []

  for _, row in cider_df.iterrows():
    idx = sorted_indexes[row['target_caption'] + '_' + row['gt_captions']]
    all_cider_scores.append(scores[idx])
  
  cider_df['target_caption_text'] = all_hypoth_captions
  cider_df['gt_caption_text'] = all_gt_captions
  cider_df['cider_score'] = all_cider_scores
  cider_df.to_csv('gdrive/Shareddrives/CS2952N/human_preference.csv',index=False)
  return avg_score

In [19]:
def compute_all_clip_scores1(image_dataloader, batch_size, model, experiment='test'):
  average_clip_score = 0

  

  cosine_sim = torch.nn.CosineSimilarity(dim=1)

  clip_df = pd.read_csv('gdrive/Shareddrives/CS2952N/human_preference.csv')


  all_clip_scores = []
  

  for idx, (image_filenames, image_features, captions) in enumerate(image_dataloader):
    
    
    print('Processing image {} of {}'.format(idx+1, len(image_dataloader)))
    
    refs = np.transpose(np.array(captions)).flatten()
    captions_per_image = np.array(captions).shape[0]

    
    text_tokens = clip.tokenize(refs).cuda()

    with torch.no_grad():
      text_features = model.encode_text(text_tokens).float()
      text_features /= text_features.norm(dim=-1, keepdim=True)

    image_features = image_features.repeat(1, captions_per_image, 1)  

    image_features = torch.reshape(image_features, (image_features.shape[0]*image_features.shape[1], image_features.shape[2]))
    
    
    
    cos_sim = cosine_sim(text_features, image_features)
    cos_sim[cos_sim<0] = 0

    clip_scores = 2.5*cos_sim


    # extract all values (for a batch) that are to be added to pandas df
    clip_scores = clip_scores.cpu().numpy()


    
    all_clip_scores += list(clip_scores)
    average_clip_score += np.sum(clip_scores)

  clip_df['clip_score'] = all_clip_scores

  clip_df.to_csv('gdrive/Shareddrives/CS2952N/human_preference.csv', index=False)
  return average_clip_score/(len(image_dataloader)*batch_size)

In [6]:
f = open(os.path.relpath('gdrive/Shareddrives/CS2952N/Flickr8k_text/ExpertAnnotations.txt'), 'r')

expert_annotations = set()

file_exists = os.path.isfile('gdrive/Shareddrives/CS2952N/human_preference.csv')

if file_exists:
  human_preference_df = pd.read_csv('gdrive/Shareddrives/CS2952N/human_preference.csv')
else:
  human_preference_df = pd.DataFrame(columns=['image_filename','target_caption','gt_captions'])

for line in tqdm(f.readlines()):
  
  data_line = line.split('\n')[0].split('\t')
  expert_annotations.add(data_line[0])
  expert_annotations.add(data_line[1].split('#')[0])

  if not file_exists:
    human_preference_df.loc[len(human_preference_df.index)] = [data_line[0], data_line[1].split('#')[0], data_line[0]] 

if not file_exists:
  human_preference_df.to_csv('gdrive/Shareddrives/CS2952N/human_preference.csv', index=False)

print('\nExpert Annotations Length: {} \n'.format(len(expert_annotations)))

100%|██████████| 5822/5822 [00:00<00:00, 538913.27it/s]


Expert Annotations Length: 1000 



In [7]:
f = open(os.path.relpath('gdrive/Shareddrives/CS2952N/Flickr8k_text/Flickr8k.token.txt'), 'r')

caption_dictionary = {}

for line in tqdm(f.readlines()):

  if line[:line.index('#')] not in expert_annotations:
    continue

  if line[:line.index('#')] not in caption_dictionary:
    caption_dictionary[line[:line.index('#')]] = [line[line.index('#')+3:].strip().lower()]
  else:
    caption_dictionary[line[:line.index('#')]].append(line[line.index('#')+3:].strip().lower())

100%|██████████| 40460/40460 [00:00<00:00, 753422.07it/s]


In [12]:
class FlickrDataset(torch.utils.data.Dataset):

  def __init__(self, human_preference_df, caption_dictionary, clip_model, image_preprocessor):

    self.BASE_PATH = 'gdrive/Shareddrives/CS2952N/Flicker8k_Dataset'

    self.human_pref_df = human_preference_df
    self.caption_dict = caption_dictionary
    self.clip_model = clip_model
    self.image_preprocessor = image_preprocessor
  
  def image_load_preprocess_embed(self, filename):

    image = Image.open(os.path.join(self.BASE_PATH, filename)).convert("RGB")
    image = self.image_preprocessor(image)

    
    image = torch.unsqueeze(image, dim=0).cuda()

    with torch.no_grad():
      image_embedding = self.clip_model.encode_image(image).float()
    image_embedding /= image_embedding.norm(dim=-1, keepdim=True)
    return image_embedding
  
  def __len__(self):
    return len(self.human_pref_df)
  
  def __getitem__(self, index):
    
    row = self.human_pref_df.iloc[index]
    image_features = self.image_load_preprocess_embed(row['image_filename'])
    captions = ['A photo depicts ' + self.caption_dict[row['target_caption']][2]]
    
   
    

    return row['image_filename'], image_features, captions

#CLIP Scores

In [20]:
dataset = FlickrDataset(human_preference_df, caption_dictionary, clip_model, preprocessor)
image_dataloader = torch.utils.data.DataLoader(dataset, batch_size=50)

avg_clip_scores = compute_all_clip_scores1(image_dataloader, 50, clip_model, experiment='flickr8k')
print(avg_clip_scores)

Processing image 1 of 117
Processing image 2 of 117
Processing image 3 of 117
Processing image 4 of 117
Processing image 5 of 117
Processing image 6 of 117
Processing image 7 of 117
Processing image 8 of 117
Processing image 9 of 117
Processing image 10 of 117
Processing image 11 of 117
Processing image 12 of 117
Processing image 13 of 117
Processing image 14 of 117
Processing image 15 of 117
Processing image 16 of 117
Processing image 17 of 117
Processing image 18 of 117
Processing image 19 of 117
Processing image 20 of 117
Processing image 21 of 117
Processing image 22 of 117
Processing image 23 of 117
Processing image 24 of 117
Processing image 25 of 117
Processing image 26 of 117
Processing image 27 of 117
Processing image 28 of 117
Processing image 29 of 117
Processing image 30 of 117
Processing image 31 of 117
Processing image 32 of 117
Processing image 33 of 117
Processing image 34 of 117
Processing image 35 of 117
Processing image 36 of 117
Processing image 37 of 117
Processing

#BLEU Score

In [ ]:
avg_bleu_scores = compute_all_bleu_scores(caption_dictionary, experiment='flickr8k')
print(avg_bleu_scores)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

[0.42205383560655263, 0.21408994031667053, 0.13163341355074143, 0.09072776977871559]


#METEOR Score

In [ ]:
avg_meteor_scores = compute_all_meteor_scores(caption_dictionary, experiment='flickr8k')
print(avg_meteor_scores)

0.26462198442989143


#SPICE Score

In [ ]:
avg_spice_scores = compute_all_spice_scores(caption_dictionary, experiment='flickr8k')
print(avg_spice_scores)

0.06270574520426106


In [ ]:
print(avg_spice_scores)

0.2305360597108907


#CIDER Score

In [ ]:
avg_cider_scores = compute_all_cider_scores(caption_dictionary, experiment='flickr8k')
print(avg_cider_scores)

0.18337528908922313
